# Author: Seunghee Kim
- Created on: 2024-12-05

In [ ]:
# Notebook Explain
# 기존에 train / valid / test 데이터셋을 구축해서 deberta-v3-xsmall 모델로 train data 학습 후 test dataset에 대해 예측했는데, 정확도 0.99가 나오게 됨.
# 데이터셋의 난이도가 너무 쉽다는 문제점 발생
# 원인 분석
# 원인: train, valid, test 를 이루고 있는 AI-Generated Text가 모두 GPT-4o-mini 모델로 구성됨.
# 즉, GPT-4o-mini 모델로 만들어진 AI-Generated Text로 학습을 했으니 그 패턴을 파악하게 돼서 GPT-4-mini 모델로 만들어진 Test dataset이 난이도가 쉬워서 정확하게 분류가 가능하게 된 것
# 해결 방안: train & valid data와 test data의 distribution을 다르게 하기 위해서, test dataset은 GPT-4o-mini Model뿐만 아니라 최대한 다양한 모델을 이용해서 구축하게 되면 Custom dataset의 난이도가 더 향상될 것이다.

In [ ]:
# train dataset_v2: Human Text 4500개, AI-Text: GPT-4o-mini Text 4500개 총 9000개
# valid dataset_v2: Human Text 450개, AI-Text: GPT-4o-mini Text 450개 총 900개

# test dataset_v2: Human Text 450개, 
# AI-Text:
# 1.'gpt-4o' 50개
# 2.'meta-llama/Llama-3.2-8B-Instruct' 50개
# 3.'Qwen/Qwen2.5-7B-Instruct' 50개
# 4.'Qwen/Qwen2.5-14B-Instruct' 50개
# 5.'Qwen/Qwen2.5-1.5B-Instruct' 50개
# 6.'Qwen/Qwen2.5-0.5B-Instruct' 50개
# 7.'gpt-3.5-turbo' 50개 
# 8.'o1-mini' 50개 
# 9.'gpt-4o-mini' 50개 
# 총 900개

# Import Library

In [21]:
import os
import pandas as pd
from tqdm import tqdm
import yaml
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoProcessor, BitsAndBytesConfig, pipeline, AutoModelForCausalLM, AutoTokenizer


# Config & Path

In [ ]:
class CFG:
    SEED = 1
    
    MODEL_1 = "gpt-4o" 
    MODEL_2 = 'meta-llama/Llama-3.1-8B-Instruct'
    MODEL_3 = 'Qwen/Qwen2.5-7B-Instruct'
    MODEL_4 = 'Qwen/Qwen2.5-14B-Instruct'
    MODEL_5 = 'Qwen/Qwen2.5-1.5B-Instruct'
    MODEL_6 = 'Qwen/Qwen2.5-0.5B-Instruct'
    MODEL_7 = 'gpt-3.5-turbo' 
    MODEL_8 = 'o1-mini' 
    MODEL_9 = 'gpt-4o-mini' 
    
    # api key가 담긴 yaml파일 (해당 yaml 파일은 .gitignore에 반드시 추가해야 함!!!)
    API_CONFIG_PATH = './config.yaml'
    
    # 1_dataset_preprocess.ipynb의 결과로 나온 전처리된 Human Dataset 경로
    DF_HUMAN_TEST_PATH = './df_human_test.csv'
    
    # AI-Generated Text까지 포함된 csv파일의 output 경로
    DF_HUMAN_AI_TEST_PATH_1 = './df_human_ai_test_1_gpt-4o.csv'
    DF_HUMAN_AI_TEST_PATH_2 = './df_human_ai_test_2_Llama-3_1-8B-Instruct.csv' 
    DF_HUMAN_AI_TEST_PATH_3 = './df_human_ai_test_3_Qwen2_5-7B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_4 = './df_human_ai_test_4_Qwen2_5-14B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_5 = './df_human_ai_test_5_Qwen2_5-1_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_6 = './df_human_ai_test_6_Qwen2_5-0_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_7 = './df_human_ai_test_7_gpt-3_5_turbo.csv'
    DF_HUMAN_AI_TEST_PATH_8 = './df_human_ai_test_8_o1-mini.csv' 
    DF_HUMAN_AI_TEST_PATH_9 = './df_human_ai_test_9_gpt-4o-mini.csv'
    
    # Final Dataset output 경로 (Human Text, AI Text, Label 존재)
    DF_FINAL_TEST_PATH = './df_final_test_v2.csv'
    
    # CACHE_DIR = '/data/ksh1234/ksh1234/huggingface_cache' 



In [23]:
# SPLIT TEST DATASET INTO 50 * 9
test_dataset = pd.read_csv(CFG.DF_HUMAN_TEST_PATH)
split_datasets = []
chunk_size = 50

for i in range(0, len(test_dataset), chunk_size):
    split_datasets.append(test_dataset.iloc[i:i + chunk_size])

test_dataset_1 = split_datasets[0]
test_dataset_2 = split_datasets[1]
test_dataset_3 = split_datasets[2]
test_dataset_4 = split_datasets[3]
test_dataset_5 = split_datasets[4]
test_dataset_6 = split_datasets[5]
test_dataset_7 = split_datasets[6]
test_dataset_8 = split_datasets[7]
test_dataset_9 = split_datasets[8]

# Model 4: Qwen/Qwen2.5-14B-Instruct

In [24]:


model_name = CFG.MODEL_4


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # load_in_4bit=True,  
    device_map="auto",  
    quantization_config=bnb_config,

)
tokenizer = AutoTokenizer.from_pretrained(model_name)

test_dataset_4['AI_Generated_Text'] = ""
test_dataset_4['Model'] = ""

for idx, row in tqdm(test_dataset_4.iterrows(), total=len(test_dataset_4)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0]  # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    grade_system_prompt = f"대한민국의 {school}학교 {grade} 학생 수준으로 답하시오. 마크다운 용법을 사용하지 말고 학생이 글을 쓰듯이 답하시오."
    messages = [
        {"role": "system", "content": grade_system_prompt},
        {"role": "user", "content": input_text},
    ]

    # 메시지를 토큰화 및 모델 입력 생성
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    llm_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    test_dataset_4.at[idx, 'AI_Generated_Text'] = llm_output
    test_dataset_4.at[idx, 'Model'] = model_name

test_dataset_4.to_csv(CFG.DF_HUMAN_AI_TEST_PATH_4, index=False)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_3138/641297606.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_4['AI_Generated_Text'] = ""
/tmp/ipykernel_3138/641297606.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_4['Model'] = ""

00%|██████████| 50/50 [28:33<00:00, 34.27s/it]

In [26]:
test_dataset_4['AI_Generated_Text']

150    반려동물을 키우는 것은 쉽지 않은 일입니다. 반려동물을 키우면서 금전적인 문제나 시...
151    반려동물을 키우는 것은 쉽지 않은 일입니다. 그래서 반려동물을 키우기 전에 잘 알아...
152    반려동물을 키우는 것은 정말 재미있지만, 그 책임감은 무겁습니다. 반려동물이 버려지...
153    반려동물을 키우는 것은 책임감이 있어야 하는 중요한 일이라고 생각해요. 반려동물을 ...
154    반려동물을 키우는 것은 정말 재미있고 즐거운 일이지만, 그 책임감도 무시할 수 없습...
155    반려동물을 키우는 것은 책임감이 있어야 하는 일이에요. 반려동물을 유기하는 건 잘못...
156    반려동물을 키우는 것은 정말 재미있고 즐거운 일이지만, 그 책임감을 잘 생각해야 한...
157    반려동물을 키우는 것은 정말 재미있고 사랑스러운 일입니다. 하지만 반려동물을 키울 ...
158    반려동물을 키우는 것은 쉽지 않은 일입니다. 그래서 반려동물을 키우기 전에 미리 준...
159    반려동물을 키우는 것은 정말 재미있고 즐거운 일이지만, 그 반려동물을 책임지는 것도...
160    반려동물을 키우는 것은 책임감이 필요하다고 생각해요. 반려동물을 그냥 귀엽다고 데려...
161    반려동물을 키우는 것은 참 좋은 일이지만, 그 책임감이 중요하다는 것을 잊으면 안 ...
162    반려동물을 키우는 것은 참 좋은 일이지만, 그 반려동물을 책임지는 것이 중요하다고 ...
163    반려동물을 키우는 것은 참 좋은 일이지만, 그 책임감도 무시할 수 없습니다. 반려동...
164    반려동물을 키우는 것은 정말 재미있지만 책임감이 있어야 한다고 생각해요. 반려동물을...
165    반려동물을 키우는 것은 쉽지 않은 일입니다. 그래서 반려동물을 키우는 사람들에게 책...
166    반려동물을 키우는 것은 아주 재미있지만 책임감이 필요하다는 것을 알아야 합니다. 반...
167    반려동물을 키우는 것은 정말 재미있고 즐거운